In [ ]:
#  #########################################################################
#  #                           IN THE NAME OF ALLAH                        #
#  #STEEL PIPE THICKNESS OPTIMIZATION ANALYSIS WITH DUCTILITY DAMAGE INDEX #
#  #                         NEWTON-RAPHSON METHOD                         #
#  #-----------------------------------------------------------------------#
#  #              THIS PROGRAM WRITTEN BY SALAR DELAVAR QASHQAI            #
#  #                   EMAIL: salar.d.ghashghaei@gmail.com                 #
#  #########################################################################

In [ ]:
#import the os module
import os
import math
import time

In [ ]:
#to create a directory at specified path with name "Data"
os.mkdir('C:\\OPENSEESPY_SALAR')
#this will create the directory with name 'Data' and will update it when we rerun the analysis,
# otherwise we have to keep deleting the old 'Data' Folder
dir = "C:\\OPENSEESPY_SALAR\\OPENSEESPY_DATA"
if not os.path.exists(dir):
    os.makedirs(dir)

In [ ]:
def HISROGRAM_BOXPLOT(X, HISTO_COLOR, LABEL):
    import numpy as np
    import matplotlib.pyplot as plt
    X = np.array(X)
    print("-------------------------")
    from scipy.stats import skew, kurtosis
    MINIMUM = np.min(X)
    MAXIMUM = np.max(X)
    #MODE = max(set(X), key=list(X).count)
    MEDIAN = np.quantile(X, .50)#q2
    MEAN = np.mean(X)
    STD = np.std(X)
    q1 = np.quantile(X, .25)
    q3 = np.quantile(X, .75)
    SKEW = skew(X)
    KURT = kurtosis(X)
    #SKEW = (MEAN - MODE) / STD
    #KURT = (np.mean((X - MEAN)**4) / STD**4)
    # Estimate confidence intervals of the output variable
    lower_bound = np.quantile(X, .05)
    upper_bound = np.quantile(X, .95)
    print("Box-Chart Datas: ")
    print(f'Minimum: {MINIMUM:.4f}')
    print(f'First quartile: {q1:.4f}')
    #print(f'Mode: {MODE:.4f}')
    print(f'Median: {MEDIAN:.4f}')
    print(f'Mean: {MEAN:.4f}')
    print(f'Std: {STD:.4f}')
    print(f'Third quartile: {q3:.4f}')
    print(f'Maximum: {MAXIMUM :.4f}')
    print(f'Skewness: {skew(X) :.4f}')
    print(f'kurtosis: {kurtosis(X) :.4f}')
    print(f"90% Confidence Interval: ({lower_bound:.4f}, {upper_bound:.4f})")
    print("-------------------------")

    plt.figure(figsize=(10,6))
    # Plot histogram of data
    count, bins, ignored = plt.hist(X, bins=100, color=HISTO_COLOR, density=True, align='mid')#, edgecolor="black"
    
    # Plot lognormal PDF
    x = np.linspace(min(bins), max(bins), 10000)
    pdf = (np.exp(-(x - MEAN)**2 / (2 * STD**2)) / (STD * np.sqrt(2 * np.pi)))
    plt.plot(x, pdf, linewidth=2, color='r', label="Normal PDF")
    
    # Plot vertical lines for risk measures
    plt.axvline(q1, color="black", linestyle="--", label=f"Quantile 0.25: {q1:.4f}")
    plt.axvline(MEDIAN, color="green", linestyle="--", label=f"Median: {MEDIAN:.4f}")
    plt.axvline(q3, color="black", linestyle="--", label=f"Quantile 0.75: {q3:.4f}")
    #plt.axvline(MODE, color="purple", linestyle="--", label=f"Mode: {MODE:.4f}")
    plt.axvline(MEAN, color="red", linestyle="--", label=f"Mean: {MEAN:.4f}")
    plt.axvline(MEAN-STD, color="blue", linestyle="--", label=f"Mean-Std: {MEAN-STD:.4f}")
    plt.axvline(MEAN+STD, color="blue", linestyle="--", label=f"Mean+Std: {MEAN+STD:.4f}")
    plt.xlabel(LABEL)
    plt.ylabel("Frequency")
    prob = np.sum(X > 0) / len(X)
    plt.title(f"Histogram - Probability of Positive {LABEL} is {100*prob:.2f} %")
    plt.legend()
    #plt.grid()
    plt.show()

    #Plot boxplot with outliers
    plt.figure(figsize=(10,6))
    plt.boxplot(X, vert=0)
    # Write the quartile data on the chart
    plt.text(q1, 1.05, f" Q1: {q1:.4f}")
    plt.text(MEDIAN, 1.1, f" Q2: {MEDIAN:.4f}")
    plt.text(q3, 1.05, f" Q3: {q3:.4f}")
    #plt.text(MODE, 1.15, f" Mode: {MODE:.4f}")
    
    #plt.text(MEAN, 0.9, f" Mean: {MEAN:.4f}")
    #plt.text(MEAN-STD, 0.9, f" Mean-Std: {MEAN-STD:.4f}")
    #plt.text(MEAN+STD, 0.9, f" Mean+Std: {MEAN+STD:.4f}")
    plt.scatter(MEAN, 1, color="red", marker="+", s=200, label=f"Mean: {MEAN:.4f}")
    plt.scatter(MEAN-STD, 1, color="green", marker="X", s=200, label=f"Mean-Std: {MEAN-STD:.4f}")
    plt.scatter(MEAN+STD, 1, color="blue", marker="*", s=200, label=f"Mean+Std:  {MEAN+STD:.4f}")
    plt.xlabel(LABEL)
    plt.ylabel("Data")
    plt.title(f"Boxplot of {LABEL}")
    plt.legend()
    plt.grid()
    plt.show()
    
# -----------------------------------------------
def PLOT_TIME_HIS(DTH, VTH, ATH, BTH):
    ## PLOT THE DATA
    import matplotlib.pyplot as plt
    # Create a 1x3 grid of subplots
    fig, axs = plt.subplots(4, 1, figsize=(14, 14))

    # Plot Displacement
    axs[0].plot(DTH, label='Displacement')
    axs[0].set_title(f'Last Analysis Displacement Time History - Max(Abs): {np.max(np.abs(DTH)):.6f}')
    axs[0].set_xlabel('Time')
    axs[0].set_ylabel('Displacement')
    axs[0].grid()

    # Plot Velocity
    axs[1].plot(VTH, label='Velocity', color='purple')
    axs[1].set_title(f'Last Analysis Velocity Time History - Max(Abs): {np.max(np.abs(ATH)):.6f}')
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel('Velocity')
    axs[1].grid()

    # Plot Acceleration
    axs[2].plot(ATH, label='Acceleration', color='green')
    axs[2].set_title(f'Last Analysis Acceleration Time History - Max(Abs): {np.max(np.abs(ATH)):.6f}')
    axs[2].set_xlabel('Time')
    axs[2].set_ylabel('Acceleration')
    axs[2].grid()
    
    # Plot Base Shear
    axs[3].plot(BTH, label='Base Shear', color='orange')
    axs[3].set_title(f'Last Analysis Base Shear Time History - Max(Abs): {np.max(np.abs(BTH)):.6f}')
    axs[3].set_xlabel('Time')
    axs[3].set_ylabel('Base Shear')
    axs[3].grid()

    # Adjust layout
    plt.tight_layout()
    plt.show()
# -----------------------------------------------    
def MAXABS_FUN(DATA_FILE, COLUMN):
    import numpy as np
    # Read and process displacement data
    NameFiles = DATA_FILE
    filename = f"{NameFiles}.txt"
    D = np.loadtxt(filename)
    #print(D)
    MAXABS = np.max(np.abs([D[:, COLUMN]]))
    #print("MAX. ABS. :", MAXABS)
    return MAXABS
# -----------------------------------------------
def PLOT_2D(X, Y, Xfit, Yfit, XLABEL, YLABEL, TITLE, COLOR, Z):
    import matplotlib.pyplot as plt
    if Z == 1:
        # Plot 1 line
        plt.figure(figsize=(12, 8))
        plt.plot(X, Y,color=COLOR)
        plt.xlabel(XLABEL)
        plt.ylabel(YLABEL)
        plt.title(TITLE)
        plt.grid(True)
        plt.show()
    if Z == 2:
        plt.plot(X, Y, Xfit, Yfit, 'r--', linewidth=3)
        plt.title(TITLE)
        plt.xlabel(XLABEL)
        plt.ylabel(YLABEL)
        plt.legend(['curve', 'bilinear fitted'], loc='lower right')
        plt.grid(True)
        plt.show()
# -----------------------------------------------
def OUTPUT_SECOND_COLUMN(X, COLUMN):
    import numpy as np
    # Time History
    filename = f"C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\\{X}.txt"
    data_collected = np.loadtxt(filename)
    X = data_collected[:, COLUMN]
    return X 
# -----------------------------------------------
def BILNEAR_CURVE(Cur, Mom):
    import numpy as np
    # bilinear fitting
    SIZE = len(Mom)
    hh = np.zeros(SIZE-1)
    Aa = np.zeros(SIZE-1)
    for i in range(SIZE-1):
        hh[i] = Cur[i+1] - Cur[i]
        Aa[i] = (Mom[i] + Mom[i+1]) * 0.5 * hh[i]

    Area = sum(Aa)
    k0 = Mom[2] / Cur[2]
    fiy = (Mom[i+1] * max(Cur) * 0.5 - Area) / (Mom[i+1] * 0.5 - k0 * max(Cur) * 0.5)
    My = k0 * fiy
    X = np.array([0, fiy, max(Cur)])
    Y = np.array([0, My, Mom[i+1]])
    """
    print('+==========================+')
    print('=   Analysis curve fitted =')
    print('  Curvature    Moment')
    print('    (1/m)      (kN.m)   ')
    print('----------------------------')
    print(np.column_stack((X.T, Y.T)))
    print('+==========================+')
    """
    # EI and Ductility_Rito of Unconfined Section
    Elastic_EI = Y[1] / X[1]
    Plastic_EI = (Y[2] - Y[1]) / (X[2] - X[1])
    Ductility_Rito = X[2] / X[1]
    Over_Stregth_Factor = Y[2] / Y[1]
    """
    print('+--------------------------------------------------------------------+')
    print(f' Elastic Flextural Rigidity :             {Elastic_EI:.2f}')
    print(f' Plastic Flextural Rigidity :             {Plastic_EI:.2f}')
    print(f' Section Ductility Ratio :                {Ductility_Rito:.2f}')
    print(f' Section  Over Stregth Factor:            {Over_Stregth_Factor:.2f}')
    print('+--------------------------------------------------------------------+')
    """
    return X, Y, Elastic_EI, Plastic_EI, Ductility_Rito, Over_Stregth_Factor

In [ ]:
# pip install openseespy

In [ ]:
# OUTPUT DATA ADDRESS:
SALAR_DIR = 'C:/OPENSEESPY_SALAR/OPENSEESPY_DATA/';

In [ ]:
#
#    ^Y
#    |
#    2       __ 
#    |          | 
#    |          |
#    |          |
#  (1)       LCol
#    |          |
#    |          |
#    |          |
#  =1=      _|_  -------->X
#

# SET UP ----------------------------------------------------------------------------


#########################################################################################################################################################################

def DYNAMIC_ANALYSIS(LR, TR, DSec, LCol, coverSec, Weight, numBarsSec, BD, fc):
    import openseespy.opensees as op
    op.wipe()
    op.model('basic', '-ndm', 2, '-ndf', 3) 
    PCol =Weight  # nodal dead-load weight per column
    g =  9810 # mm/s^2
    Mass =  PCol/g

    L = LCol - LR; # NOT Retrofitrd Length
    # nodal coordinates:
    op.node(1, 0.0, 0.0) # node#, X, Y
    op.node(2, 0.0, LR)
    op.node(3, 0.0, LCol)
    # Single point constraints -- Boundary Conditions
    op.fix(1, 1, 1, 1) # node DX DY RZ
    # node#, Mx My Mz, Mass=Weight/g, neglect rotational inertia at nodes
    op.mass(2, Mass, 1e-9, 0.0)
    
    barAreaSec = (3.1415 * BD**2) / 4  # area of longitudinal-reinforcement bars
    
    ColSecTag01 = 1			# Retrofited Section - assign a tag number to the column section
    ColSecTag02 = 2			# Not Retrofited Section - assign a tag number to the column section
    # MATERIAL parameters -------------------------------------------------------------------
    IDconcCore = 1; 				# material ID tag -- confined core concrete
    IDconcCover = 2; 				# material ID tag -- unconfined cover concrete
    IDreinf = 3; 				# material ID tag -- reinforcement
    IDrp = 4; 				# material ID tag -- retofited plate 
    
    # nominal concrete compressive strength
    Ec = 4700 * math.sqrt(-fc) # Concrete Elastic Modulus (the term in sqr root needs to be in psi

    # confined concrete
    Kfc = 1.3;			# ratio of confined to unconfined concrete strength
    fc1C = Kfc*fc;		# CONFINED concrete (mander model), maximum stress
    eps1C = 2*fc1C/Ec;	# strain at maximum stress 
    fc2C = 0.2*fc1C;		# ultimate stress
    eps2C = 5*eps1C;		# strain at ultimate stress 
    # unconfined concrete
    fc1U = fc;			# UNCONFINED concrete (todeschini parabolic model), maximum stress
    eps1U = -0.0025;			# strain at maximum strength of unconfined concrete
    fc2U = 0.2*fc1U;		# ultimate stress
    eps2U = -0.012;			# strain at ultimate stress
    Lambda = 0.1;				# ratio between unloading slope at $eps2 and initial slope $Ec
    # tensile-strength properties
    ftC = -0.55*fc1C;		# tensile strength +tension
    ftU = -0.55*fc1U;		# tensile strength +tension
    Ets = ftU/0.002;		# tension softening stiffness
    # REBAR MATERIAL PROPERTIES:
    Fy = 4000			# Steel rebar yield stress
    Cy = 0.02			# Steel rebar yield strain
    Es = Fy/Cy				# modulus of steel
    Bs = 0.01				# strain-hardening ratio 
    R0 = 18.0				# control the transition from elastic to plastic branches
    cR1 = 0.925				# control the transition from elastic to plastic branches
    cR2 = 0.15				# control the transition from elastic to plastic branches
    # PLATE MATERIAL PROPERTIES:
    FyP = 2400			# Steel plate yield stress
    CyP = 0.02			# Steel plate yield strain
    EsP = FyP/CyP				# modulus of steel
    BsP = 0.01				# strain-hardening ratio 
    R0P = 18.0				# control the transition from elastic to plastic branches
    cR1P = 0.925				# control the transition from elastic to plastic branches
    cR2P = 0.15				# control the transition from elastic to plastic branches

    op.uniaxialMaterial('Concrete02', IDconcCore, fc1C, eps1C, fc2C, eps2C, Lambda, ftC, Ets) # build cover concrete (confined)
    op.uniaxialMaterial('Concrete02', IDconcCover, fc1U, eps1U, fc2U, eps2U, Lambda, ftU, Ets) # build cover concrete (unconfined)
    op.uniaxialMaterial('Steel02', IDreinf, Fy, Es, Bs, R0,cR1,cR2) # build reinforcement material
    op.uniaxialMaterial('Steel02', IDrp, FyP, EsP, BsP, R0P,cR1P,cR2P) # build retrofited plate material
    # FIBER SECTION properties -------------------------------------------------------------
    ri = 0.0;			# inner radius of the section, only for hollow sections
    ro = (DSec + TR)/2;	# overall (outer) radius of the section
    rc = ro - coverSec - TR;					# Core radius
    rt = ro - TR;					# Plate radius
    nfCoreR  = 8;		# number of radial divisions in the core (number of "rings")
    nfCoreT = 8;		# number of theta divisions in the core (number of "wedges")
    nfCoverR = 4;		# number of radial divisions in the cover
    nfCoverT = 8;		# number of theta divisions in the cover
    nfCoverRpl = 4;		# number of radial divisions in the plate
    nfCoverTpl = 8;		# number of theta divisions in the plate
    # -----------------------------------------------------------------------------------------------------
    # RETROFITTED SECTION: 
    op.section('Fiber', ColSecTag01)
    # Define the core patch
    op.patch('circ', IDconcCore, nfCoreT, nfCoreR, 0, 0, ri, rc, 0, 360)
    # Define the four cover patches
    op.patch('circ', IDconcCover, nfCoverT, nfCoverR, 0, 0, rc, ro, 0, 360)
    # Define the four plate patches
    op.patch('circ', IDrp, nfCoverTpl, nfCoverRpl, 0, 0, rt, ro, 0, 360)
    # Define reinfocement layers 
    theta = 360.0/numBarsSec; # Determine angle increment between bars
    # Define the reinforcing layer
    op.layer('circ', IDreinf, numBarsSec, barAreaSec, 0, 0, rc, theta, 360)
    
    ColTransfTag01 = 1
    op.geomTransf('Linear', ColTransfTag01)
    numIntgrPts = 5
    eleTag = 1
    op.element('nonlinearBeamColumn', eleTag, 1, 2, numIntgrPts, ColSecTag01, ColTransfTag01)
    # -----------------------------------------------------------------------------------------------------
    # NOT RETROFITTED SECTION:
    op.section('Fiber', ColSecTag02)
    # Define the core patch
    op.patch('circ', IDconcCore, nfCoreT, nfCoreR, 0, 0, ri, rc, 0, 360)
    # Define the cover patches
    op.patch('circ', IDconcCover, nfCoverT, nfCoverR, 0, 0, rc, ro, 0, 360)
    # Define reinfocement layers 
    theta = 360.0/numBarsSec;		# Determine angle increment between bars
    # Define the reinforcing layer
    op.layer('circ', IDreinf, numBarsSec, barAreaSec, 0, 0, rc, theta, 360)
    
    ColTransfTag02 = 2
    op.geomTransf('Linear', ColTransfTag02)
    numIntgrPts = 5
    eleTag = 2
    op.element('nonlinearBeamColumn', eleTag, 2, 3, numIntgrPts, ColSecTag02, ColTransfTag02)
    # -----------------------------------------------------------------------------------------------------

    #import InelasticFiberSection
    op.recorder('EnvelopeNode','-file', f"{SALAR_DIR}MD.txt" ,'-time','-node',2,'-dof',1,'disp');# max. displacements of free nodes 2
    op.recorder('EnvelopeNode','-file', f"{SALAR_DIR}MD02.txt" ,'-time','-node',3,'-dof',1,'disp');# max. displacements of free nodes 3
    op.recorder('EnvelopeNode','-file',f"{SALAR_DIR}MV.txt" ,'-time','-node',3,'-dof',1,'disp');# max. vel of free nodes 3
    op.recorder('EnvelopeNode','-file', f"{SALAR_DIR}MA.txt" ,'-time','-node',3,'-dof',1,'disp');# max. accel of free nodes 3	
    op.recorder('Node', '-file', f"{SALAR_DIR}DTH.txt",'-time', '-node', 3, '-dof', 1,2,3, 'disp')# Displacement Time History nodes 3
    op.recorder('Node', '-file', f"{SALAR_DIR}DTH02.txt",'-time', '-node', 2, '-dof', 1,2,3, 'disp')# Displacement Time History nodes 2
    op.recorder('Node', '-file', f"{SALAR_DIR}VTH.txt",'-time', '-node', 2, '-dof', 1,2,3, 'vel')# Vel Time History nodes 2
    op.recorder('Node', '-file', f"{SALAR_DIR}ATH.txt",'-time', '-node', 2, '-dof', 1,2,3, 'accel')# Accel Time History nodes 2
    op.recorder('Node', '-file', f"{SALAR_DIR}BTH.txt",'-time', '-node', 1, '-dof', 1,2,3, 'reaction')# Base Shear Time History nodes 1
    #op.recorder('Element', '-file', f"{SALAR_DIR}FCol.txt",'-time', '-ele', 1, 'section', 1, 'globalForce')
    #op.recorder('Element', '-file', f"{SALAR_DIR}ForceColSec1.txt",'-time', '-ele', 1, 'section', 1, 'force')
    op.recorder('Element', '-file', f"{SALAR_DIR}DCol.txt",'-time', '-ele', 1, 'section', 1, 'deformations')# Rotation Curvature Time History nodes 1
    #op.recorder('Element', '-file', f"{SALAR_DIR}stCol.txt",'-time', '-ele', 1, 'section', 1,'fiber',ro, 0,IDreinf, 'stressStrain')# Stress and Strain Time History nodes 1

    #defining gravity loads
    op.timeSeries('Linear', 1)
    op.pattern('Plain', 1, 1)
    op.load(2, 0.0, -PCol, 0.0)

    Tol = 1e-8 # convergence tolerance for test
    NstepGravity = 10
    DGravity = 1/NstepGravity
    op.integrator('LoadControl', DGravity) # determine the next time step for an analysis
    op.numberer('Plain') # renumber dof's to minimize band-width (optimization), if you want to
    op.system('BandGeneral') # how to store and solve the system of equations in the analysis
    op.constraints('Plain') # how it handles boundary conditions
    op.test('NormDispIncr', Tol, 6) # determine if convergence has been achieved at the end of an iteration step
    op.algorithm('Newton') # use Newton's solution algorithm: updates tangent stiffness at every iteration
    op.analysis('Static') # define type of analysis static or transient
    op.analyze(NstepGravity) # apply gravity

    op.loadConst('-time', 0.0) #maintain constant gravity loads and reset time to zero

    #applying Dynamic Ground motion analysis
    GMdirection = 1
    GMfile = 'BM68elc.acc'
    GMfact = 1.0



    Lambda01 = op.eigen('-fullGenLapack', 1) # eigenvalue mode 1
    Lambda02 = op.eigen('-genBandArpack', 1) # eigenvalue mode 1
    #print(Lambda)
    Omega = math.pow(max(min(Lambda01), min(Lambda02)), 0.5)
    betaKcomm = 2 * (0.02/Omega)

    xDamp = 0.02				# 2% damping ratio
    alphaM = 0.0				# M-prop. damping; D = alphaM*M	
    betaKcurr = 0.0		# K-proportional damping;      +beatKcurr*KCurrent
    betaKinit = 0.0 # initial-stiffness proportional damping      +beatKinit*Kini

    op.rayleigh(alphaM,betaKcurr, betaKinit, betaKcomm) # RAYLEIGH damping

    # Uniform EXCITATION: acceleration input
    IDloadTag = 400			# load tag
    dt = 0.01			# time step for input ground motion
    GMfatt = 1.0			# data in input file is in g Unifts -- ACCELERATION TH
    maxNumIter = 10
    op.timeSeries('Path', 2, '-dt', dt, '-filePath', GMfile, '-factor', GMfact)
    op.pattern('UniformExcitation', IDloadTag, GMdirection, '-accel', 2) 

    op.wipeAnalysis()
    op.constraints('Transformation')
    op.numberer('Plain')
    op.system('BandGeneral')
    op.test('EnergyIncr', Tol, maxNumIter)
    op.algorithm('ModifiedNewton')

    NewmarkGamma = 0.5
    NewmarkBeta = 0.25
    op.integrator('Newmark', NewmarkGamma, NewmarkBeta)
    op.analysis('Transient')

    DtAnalysis = 0.01
    TmaxAnalysis = 10.0

    Nsteps =  int(TmaxAnalysis/ DtAnalysis)

    ok = op.analyze(Nsteps, DtAnalysis)

    tCurrent = op.getTime()

    # for gravity analysis, load control is fine, 0.1 is the load factor increment (http://opensees.berkeley.edu/wiki/index.php/Load_Control)

    test = {1:'NormDispIncr', 2: 'RelativeEnergyIncr', 4: 'RelativeNormUnbalance',5: 'RelativeNormDispIncr', 6: 'NormUnbalance'}
    algorithm = {1:'KrylovNewton', 2: 'SecantNewton' , 4: 'RaphsonNewton',5: 'PeriodicNewton', 6: 'BFGS', 7: 'Broyden', 8: 'NewtonLineSearch'}

    for i in test:
        for j in algorithm:

            if ok != 0:
                if j < 4:
                    op.algorithm(algorithm[j], '-initial')

                else:
                    op.algorithm(algorithm[j])

                op.test(test[i], Tol, 1000)
                ok = op.analyze(Nsteps, DtAnalysis)                            
                print(test[i], algorithm[j], ok)             
                if ok == 0:
                    break
            else:
                continue

    #u2 = op.nodeDisp(2, 1)
    #print("u2 = ", u2)
    print('Ground Motion Done.')
    op.wipe()
    

In [ ]:
### -------------------------------
###    MOMENT-CURVATURE FUNCTION
### -------------------------------

def MC(P, DR, numIncr, TR, DSec, coverSec, numBarsSec, BD, fc, TOLERANCE, MAX_ITERATION):
    import openseespy.opensees as op
    def MomentCurvature(ColSecTag, axialLoad, maxK, numIncr=100):

        # Define two nodes at (0,0)
        op.node(1, 0.0, 0.0)
        op.node(2, 0.0, 0.0)

        # Fix all degrees of freedom except axial and bending
        op.fix(1, 1, 1, 1)
        op.fix(2, 0, 1, 0)

        # Define element
        #                             tag ndI ndJ  secTag
        op.element('zeroLengthSection',  1,   1,   2,  ColSecTag)
        # Create recorder
        op.recorder('Node', '-file', f"{SALAR_DIR}CUR.txt",'-time', '-node', 2, '-dof', 3, 'disp')# Curvature Time History nodes 2
        op.recorder('Node', '-file', f"{SALAR_DIR}MOM.txt",'-time', '-node', 1, '-dof', 3, 'reaction')# Base Shear Time History nodes 1
        op.recorder('Element','-ele',1,'-file',f'{SALAR_DIR}fiberA_StressStrain.txt','section','fiber',(DSec+TR)/2,0,'stressStrain')# Compression fiber
        op.recorder('Element','-ele',1,'-file',f'{SALAR_DIR}fiberB_StressStrain.txt','section','fiber',-(DSec+TR)/2,0,'stressStrain')# Tension fiber
        op.recorder('Element','-ele',1,'-file',f'{SALAR_DIR}fiberC_StressStrain.txt','section','fiber',(DSec+TR)/2 - TR - coverSec,0,'stressStrain')# Top fiber - steel
        op.recorder('Element','-ele',1,'-file',f'{SALAR_DIR}fiberD_StressStrain.txt','section','fiber',-(DSec+TR)/2 + TR + coverSec,0,'stressStrain')# Bottom fiber - steel
        
        # Define constant axial load
        op.timeSeries('Constant', 1)
        op.pattern('Plain', 1, 1)
        op.load(2, axialLoad, 0.0, 0.0)

        # Define analysis parameters
        op.integrator('LoadControl', 0.0)
        op.system('SparseGeneral', '-piv')
        op.test('NormUnbalance', TOLERANCE, MAX_ITERATION)
        op.numberer('Plain')
        op.constraints('Plain')
        op.algorithm('Newton')
        op.analysis('Static')

        # Do one analysis for constant axial load
        op.analyze(1)

        # Define reference moment
        op.timeSeries('Linear', 2)
        op.pattern('Plain',2, 2)
        op.load(2, 0.0, 0.0, 1.0)

        # Compute curvature increment
        dK = maxK / numIncr

        # Use displacement control at node 2 for section analysis
        op.integrator('DisplacementControl', 2,3,dK,1,dK,dK)

        # Do the section analysis
        op.analyze(numIncr)


    op.wipe()
    
    ####      Start Moment Curvature Analysis
    barAreaSec = (3.1415 * BD**2) / 4
    # Define model builder
    # --------------------
    op.model('basic','-ndm',2,'-ndf',3)
    ColSecTag = 1
    # MATERIAL parameters -------------------------------------------------------------------
    IDconcCore = 1; 				# material ID tag -- confined core concrete
    IDconcCover = 2; 				# material ID tag -- unconfined cover concrete
    IDreinf = 3; 				# material ID tag -- reinforcement
    IDrp = 4; 				# material ID tag -- retofited plate 
    # nominal concrete compressive strength
    Ec = 4700 * math.sqrt(-fc) # Concrete Elastic Modulus (the term in sqr root needs to be in psi

    # confined concrete
    Kfc = 1.3;			# ratio of confined to unconfined concrete strength
    fc1C = Kfc*fc;		# CONFINED concrete (mander model), maximum stress
    eps1C = 2*fc1C/Ec;	# strain at maximum stress 
    fc2C = 0.2*fc1C;		# ultimate stress
    eps2C = 5*eps1C;		# strain at ultimate stress 
    # unconfined concrete
    fc1U = fc;			# UNCONFINED concrete (todeschini parabolic model), maximum stress
    eps1U = -0.0025;			# strain at maximum strength of unconfined concrete
    fc2U = 0.2*fc1U;		# ultimate stress
    eps2U = -0.012;			# strain at ultimate stress
    Lambda = 0.1;				# ratio between unloading slope at $eps2 and initial slope $Ec
    # tensile-strength properties
    ftC = -0.55*fc1C;		# tensile strength +tension
    ftU = -0.55*fc1U;		# tensile strength +tension
    Ets = ftU/0.002;		# tension softening stiffness
    # REBAR MATERIAL PROPERTIES:
    Fy = 4000			# Steel rebar yield stress
    Cy = 0.02			# Steel rebar yield strain
    Es = Fy/Cy				# modulus of steel
    Bs = 0.01				# strain-hardening ratio 
    R0 = 18.0				# control the transition from elastic to plastic branches
    cR1 = 0.925				# control the transition from elastic to plastic branches
    cR2 = 0.15				# control the transition from elastic to plastic branches
    # PLATE MATERIAL PROPERTIES:
    FyP = 2400			# Steel plate yield stress
    CyP = 0.02			# Steel plate yield strain
    EsP = FyP/CyP				# modulus of steel
    BsP = 0.01				# strain-hardening ratio 
    R0P = 18.0				# control the transition from elastic to plastic branches
    cR1P = 0.925				# control the transition from elastic to plastic branches
    cR2P = 0.15				# control the transition from elastic to plastic branches

    op.uniaxialMaterial('Concrete02', IDconcCore, fc1C, eps1C, fc2C, eps2C, Lambda, ftC, Ets) # build cover concrete (confined)
    op.uniaxialMaterial('Concrete02', IDconcCover, fc1U, eps1U, fc2U, eps2U, Lambda, ftU, Ets) # build cover concrete (unconfined)
    op.uniaxialMaterial('Steel02', IDreinf, Fy, Es, Bs, R0,cR1,cR2) # build reinforcement material
    op.uniaxialMaterial('Steel02', IDrp, FyP, EsP, BsP, R0P,cR1P,cR2P) # build retrofited plate material
    # FIBER SECTION properties -------------------------------------------------------------
    ri = 0.0;			# inner radius of the section, only for hollow sections
    ro = (DSec + TR)/2;	# overall (outer) radius of the section
    rc = ro - coverSec - TR;					# Core radius
    rt = ro - TR;					# Plate radius
    nfCoreR  = 8;		# number of radial divisions in the core (number of "rings")
    nfCoreT = 8;		# number of theta divisions in the core (number of "wedges")
    nfCoverR = 4;		# number of radial divisions in the cover
    nfCoverT = 8;		# number of theta divisions in the cover
    nfCoverRpl = 4;		# number of radial divisions in the plate
    nfCoverTpl = 8;		# number of theta divisions in the plate
    # -----------------------------------------------------------------------------------------------------
    # RETROFITTED SECTION: 
    op.section('Fiber', ColSecTag)
    # Define the core patch
    op.patch('circ', IDconcCore, nfCoreT, nfCoreR, 0, 0, ri, rc, 0, 360)
    # Define the four cover patches
    op.patch('circ', IDconcCover, nfCoverT, nfCoverR, 0, 0, rc, ro, 0, 360)
    # Define the four plate patches
    op.patch('circ', IDrp, nfCoverTpl, nfCoverRpl, 0, 0, rt, ro, 0, 360)
    # Define reinfocement layers 
    theta = 360.0/numBarsSec; # Determine angle increment between bars
    # Define the reinforcing layer
    op.layer('circ', IDreinf, numBarsSec, barAreaSec, 0, 0, rc, theta, 360)

    # -----------------------------------------------------------------------------------------------------
    # set yield  Curvature
    Ky = CyP / (0.5 * DSec)
    #print('Ky', Ky)

    # set ultimate Curvature
    Ku = Ky * DR
    #print('Ku', Ku)


    # Call the section analysis procedure:
    MomentCurvature(ColSecTag, P, Ku, numIncr)
    op.wipe()



In [ ]:
### ---------------------------------------------------------------------
###    MOMENT-CURVATURE ANALYSIS COMPOSITE SECTION COLUMN PIPE THICKNESS
### ---------------------------------------------------------------------

P = -100000.0 # [N] Set axial load
DR = 1.5 # set ductility ratio for moment curvature
TOLERANCE = 1e-9 # Converganve tolerance for moment curvature
MAX_ITERATION = 1000000 # Maximum Tolerance for moment curvature

numIncr = 100# Number of analysis increments
TR = 1.0 # [mm] Retrofit Column Section Pipe Thickness
DSec = 500 # [mm] Column Diameter
coverCol = 50.0   # [mm] Column cover to reinforcing steel NA.
numBarsCol = 12  # number of longitudinal-reinforcement bars in column. (symmetric top & bot)
BD = 18  # [mm] Rebar Diamater
fc = -25.0 # [N/mm^2] Concrete Compressive Strength (+Tension, -Compression)


## Analysis Moment-Curvature
MC(P, DR, numIncr, TR, DSec, coverCol, numBarsCol, BD, fc, TOLERANCE, MAX_ITERATION)

CUR = OUTPUT_SECOND_COLUMN('CUR', 1)
MOM = OUTPUT_SECOND_COLUMN('MOM', 1)

xx, yy, Elastic_EI, Plastic_EI, Ductility_Rito, Over_Stregth_Factor = BILNEAR_CURVE(CUR, -MOM)
PLOT_2D(CUR, -MOM, xx, yy, XLABEL='Curvature', YLABEL='Moment', TITLE='Moment-Curvature Analysis', COLOR='black',Z = 2)
DI = (9.28571000e-08 - xx[1]) / (xx[2] - xx[1])
print('Ductility Damage Index: ', DI)

In [ ]:
### PLOT STRESS AND STRAIN RELATION OF FIBERS
fiberA_stress = OUTPUT_SECOND_COLUMN('fiberA_StressStrain', 0)
fiberA_strain = OUTPUT_SECOND_COLUMN('fiberA_StressStrain', 1)
fiberB_stress = OUTPUT_SECOND_COLUMN('fiberB_StressStrain', 0)
fiberB_strain = OUTPUT_SECOND_COLUMN('fiberB_StressStrain', 1)
fiberC_stress = OUTPUT_SECOND_COLUMN('fiberC_StressStrain', 0)
fiberC_strain = OUTPUT_SECOND_COLUMN('fiberC_StressStrain', 1)
fiberD_stress = OUTPUT_SECOND_COLUMN('fiberD_StressStrain', 0)
fiberD_strain = OUTPUT_SECOND_COLUMN('fiberD_StressStrain', 1)

import matplotlib.pyplot as plt

# Create subplots
fig, ax = plt.subplots(figsize=(10, 6))

# Plot stress-strain curves for each fiber
ax.plot(fiberA_strain, fiberA_stress, label='Fiber A')
ax.plot(fiberB_strain, fiberB_stress, label='Fiber B')
ax.plot(fiberC_strain, fiberC_stress, label='Fiber C')
ax.plot(fiberD_strain, fiberD_stress, label='Fiber D')
ax.set_title('Stress-Strain Curves for Different Fibers')
ax.set_xlabel('Strain')
ax.set_ylabel('Stress')
ax.legend()
ax.grid()
plt.show()


In [ ]:
### ---------------------------------------------------------------------
###    COMPOSITE SECTION COLUMN PIPE THICKNESS OPTIMIZATION
### ---------------------------------------------------------------------

# define section geometry
LR = 1000.0 # [mm] Retrofit Column length
TR = 1.0 # [mm] Retrofit Column Section Pipe Thickness

DR = 1.5 # set ductility ratio for moment curvature
LCol = 3000.0 # [mm] Column length
DSec = 300 # [mm] Column Diameter
coverCol = 50.0   # [mm] Column cover to reinforcing steel NA.
Weight = 10000.0 # [N] superstructure weight
numBarsCol = 12  # number of longitudinal-reinforcement bars in column. (symmetric top & bot)
BD = 18  # [mm] Rebar Diamater
fc = -25.0 # [N/mm^2] Concrete Compressive Strength (+Tension, -Compression)
st = 5 # Sleep Time 

X = TR # Intial Guess for Retrofit Pipe Thickness

ESP = 1e-3 # Finite difference derivative Convergence Tolerance
TOLERANCE = 1e-4 # Convergence Tolerance
RESIDUAL = 100 # Convergence Residual 
IT = 0 # Intial Iteration
ITMAX = 100000 # Max. Iteration
TARGET_DAMAGE_INDEX = -0.1 # Target Section Ductility Damage Index
## NEGETIVE DEMAND DUCTILITY DAMAGE INDEX, MEANS YOU LOOK FOR DEMAND DUCTILITY SAFETY INDEX

DATA_FILE ='C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\DCol'  # DISPLACEMENT TIME HISTORY

# monitor cpu time
import time
starttime = time.process_time()

### FIND THE OPTIMUM VALUE 
while (RESIDUAL > TOLERANCE):
    MC(-Weight, DR, numIncr, X, DSec, coverCol, numBarsCol, BD, fc, TOLERANCE, MAX_ITERATION)
    DYNAMIC_ANALYSIS(LR, X, DSec, LCol, coverCol, Weight, numBarsCol, BD, fc)
    time.sleep(st);# Sleep time
    CUR = OUTPUT_SECOND_COLUMN('CUR', 1)
    MOM = OUTPUT_SECOND_COLUMN('MOM', 1)
    xx, yy, _, _, _, _ = BILNEAR_CURVE(CUR, -MOM)
    DEMAND_CURVATURE = MAXABS_FUN(DATA_FILE, 1)
    DI = (DEMAND_CURVATURE - xx[1]) / (xx[2] - xx[1]) # Target Section Ductility Damage Index
    print('DI', DI)
    F = DI - TARGET_DAMAGE_INDEX
    
    # Evaluate at Xmain and Fmin
    Xmin = X - ESP
    MC(-Weight, DR, numIncr, Xmin, DSec, coverCol, numBarsCol, BD, fc, TOLERANCE, MAX_ITERATION)
    DYNAMIC_ANALYSIS(LR, Xmin, DSec, LCol, coverCol, Weight, numBarsCol, BD, fc)
    time.sleep(st);# Sleep time
    CUR = OUTPUT_SECOND_COLUMN('CUR', 1)
    MOM = OUTPUT_SECOND_COLUMN('MOM', 1)
    xx, yy, _, _, _, _ = BILNEAR_CURVE(CUR, -MOM)
    DEMAND_CURVATURE = MAXABS_FUN(DATA_FILE, 1)
    DI = (DEMAND_CURVATURE - xx[1]) / (xx[2] - xx[1]) # Target Section Ductility Damage Index
    print('DI', DI)
    Fmin = DI - TARGET_DAMAGE_INDEX
    #print('Fmin02: ', Fmin02)
    
    # Evaluate at Xmax and Fmax
    Xmax = X + ESP
    MC(-Weight, DR, numIncr, Xmax, DSec, coverCol, numBarsCol, BD, fc, TOLERANCE, MAX_ITERATION)
    DYNAMIC_ANALYSIS(LR, Xmax, DSec, LCol, coverCol, Weight, numBarsCol, BD, fc)
    time.sleep(st);# Sleep time
    CUR = OUTPUT_SECOND_COLUMN('CUR', 1)
    MOM = OUTPUT_SECOND_COLUMN('MOM', 1)
    xx, yy, _, _, _, _ = BILNEAR_CURVE(CUR, -MOM)
    DEMAND_CURVATURE = MAXABS_FUN(DATA_FILE, 1)
    DI = (DEMAND_CURVATURE - xx[1]) / (xx[2] - xx[1]) # [1/mm] Target Section Ductility Damage Index
    Fmax = DI - TARGET_DAMAGE_INDEX
    #print('Fmax02: ', Fmax02)
    # Calculate the Finite difference derivative of F
    DF = (Fmax - Fmin)/(2 * ESP)
    #print('DF: ', DF)
    # Calculate dx
    DX = F / DF
    #print('DX: ', DX)
    # Calculate residual
    RESIDUAL = abs(DX)
    print('RESIDUAL: ', RESIDUAL)
    X -= DX # update X2 (Pipe Thickness)
    IT += 1 # update iteration: 
    if IT == ITMAX:
        print("\t\t Iteration reached to Max. Iteration")
        print("\t\t Change ESP and TOLERANCE for better Convergence")
        X = -1;
        break;
    if RESIDUAL < TOLERANCE:
        print(f'\t\t Optimum Pipe Thickness: {X:.4f}')
        print(f'\t\t Iteration Counts:       {IT}')
        print(f'\t\t Convergence Residual:   {RESIDUAL:.10e}')
        
        
    #print(X)
    
totaltime = time.process_time() - starttime
print(f'\nTotal time (s): {totaltime:.4f} \n\n')

In [ ]:
### LOAD DATA
import numpy as np

# Displacement Time History
DTH = OUTPUT_SECOND_COLUMN('DTH02', 1)
# Velocity Time History
VTH = OUTPUT_SECOND_COLUMN('VTH', 1)
# Acceleration Time History
ATH = OUTPUT_SECOND_COLUMN('ATH', 1)
# Base Shear Time History
BTH = OUTPUT_SECOND_COLUMN('BTH', 1)

print(len(DTH), len(VTH),len(ATH),len(BTH))


In [ ]:
HISROGRAM_BOXPLOT(DTH, HISTO_COLOR='blue', LABEL='Displacement Time History')

In [ ]:
HISROGRAM_BOXPLOT(VTH, HISTO_COLOR='purple', LABEL='Velocity Time History')

In [ ]:
HISROGRAM_BOXPLOT(ATH, HISTO_COLOR='green', LABEL='Acceleration Time History')

In [ ]:
HISROGRAM_BOXPLOT(BTH, HISTO_COLOR='orange', LABEL='Base Shear Time History')

In [ ]:
# PLOT TIME HISTORY 
PLOT_TIME_HIS(DTH, VTH, ATH, BTH)

In [ ]:
PLOT_2D(DTH, BTH,_,_, XLABEL='Displacement Time History', YLABEL='Base Shear Time History', TITLE='Base Shear and Displacement Time History', COLOR='black', Z= 1)    